Task 2 TSV to JSON Scripts

In [ ]:
import pandas as pd
import json

In [ ]:
#code to remove redundant columns
df = pd.read_csv('../data/tsvOriginal/BFROFinal.tsv', sep='\t')
filtered_columns = [col for col in df.columns if 'Report_Entity' not in col and 'Observed_Entity' not in col and 'Also_Noticed_Entity' not in col]
data = df[filtered_columns]
# data.to_csv('new_data.tsv', sep='\t', index=False)

/var/folders/nw/tjpcs72n3p50mg20k2vgl11w0000gn/T/ipykernel_2127/2084665016.py:2: DtypeWarning: Columns (135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,171,172,173,174,175,176,177,178,179,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../data/tsvOriginal/BFROFinal.tsv', sep='\t')


In [ ]:
#data for grouped bar chart
class_agg = data.groupby(["State", "Class"]).size().reset_index(name="count")
class_agg.to_json("../data/class_agg.json", orient="records")

In [ ]:
#data for sightings and witnesses over time
sightings = data.groupby('yr').size().reset_index(name='TotalSightings')
avg_witness_count = data.groupby('yr')['WitnessCount'].mean().reset_index()
over_time = pd.merge(sightings, avg_witness_count, on='yr', how='left')
over_time.rename(columns={'WitnessCount': 'AvgWitnessCount'}, inplace=True)
over_time = over_time.sort_values(by = 'TotalSightings', ascending = False)
over_time = over_time.head(30)
over_time = over_time.sort_values(by = 'yr', ascending = True)
over_time = over_time.to_json(orient='records')

with open('../data/over_time.json', 'w') as file:
    file.write(over_time)

In [ ]:
# getting numebr of sightings per State
StateAndSighting = data['State'].value_counts().reset_index()
StateAndSighting.columns = ['State', 'Sightings']
stateandsightingsjson = StateAndSighting.to_json(orient='records')
with open('../data/sighting.json', 'w') as f:
    f.write(stateandsightingsjson)

In [ ]:
# Got lat/lng for each state and combined it with the sightings json
#downloaded lat/lng from https://gist.github.com/meiqimichelle/7727723
sightings_data = pd.read_json('../data/sighting.json')
geo_data = pd.read_json('../data/USstates_avg_latLong.json')
combined_data = pd.merge(sightings_data, geo_data, how='left', left_on='State', right_on='state')
combined_data.head()
stateandsightingsgeojson = combined_data.to_json(orient='records')
with open('../data/SightingCountandGeo.json', 'w') as f:
    f.write(stateandsightingsgeojson)

In [ ]:
# getting bigfoot hotspot info
top10locations = data.groupby(['State', 'County']).size().nlargest(10).reset_index()[['State', 'County']]
data['BigFoot Hotspot'] = data.apply(lambda x: (x['State'], x['County']) in [tuple(x) for x in top10locations.values], axis=1)
bigfoothotspotsjson = data[['State', 'County', 'BigFoot Hotspot']].to_json(orient='records')
bigfoot_hotspots_file = '../data/bigfoothotspots.json'
with open(bigfoot_hotspots_file, 'w') as f:
    f.write(bigfoothotspotsjson)

/var/folders/nw/tjpcs72n3p50mg20k2vgl11w0000gn/T/ipykernel_2127/2542045105.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['BigFoot Hotspot'] = data.apply(lambda x: (x['State'], x['County']) in [tuple(x) for x in top10locations.values], axis=1)


In [ ]:
#getting witness counts per state by season
Witnessdf = data.dropna(subset=['State', 'Season', 'WitnessCount'])
Witnessdf = Witnessdf[(Witnessdf['Season'] != 'Unknown') & (Witnessdf['WitnessCount'] > 1)]
WitnessJson = Witnessdf[['State', 'Season', 'Class', 'WitnessCount']].to_json(orient='records')
WitnessJsonFile = '../data/Witnesses.json'
with open(WitnessJsonFile, 'w') as f:
    f.write(WitnessJson)

In [ ]:
#grouping by class, month, closest park, and severity
df = data[['Class', 'mth', 'closestPark', 'Severity']]
df = df[~df['Class'].isnull()]
df = df[~df['closestPark'].isnull()]
df = df[~df['Severity'].isnull()]
df.head(30)
#count is 4059)

,Class,mth,closestPark,Severity
22,Class A,September,gulf islands ns,Light
23,Class A,January,tuskegee institute nhs,Light
25,Class B,September,little river canyon npres,Light
26,Class A,June,little river canyon npres,Light
27,Class B,September,little river canyon npres,Light
28,Class C,June,little river canyon npres,Light
30,Class A,June,little river canyon npres,Severe
36,Class A,July,gulf islands ns,Light
37,Class B,October,gulf islands ns,Light
38,Class B,September,gulf islands ns,Light


In [ ]:
# Group the data by the features
grouped_data = df.groupby(["closestPark", "Class", "mth", "Severity"]).size().reset_index(name="count")
# Define a function to convert DataFrame to hierarchical JSON
def df_to_json(df):
    data = {"name": "root", "children": []}
    for _, row in df.iterrows():
        current_node = data
        for feature in ["closestPark", "Class", "mth", "Severity"]:
            value = row[feature]
            children = current_node.get("children", [])
            child = next((c for c in children if c["name"] == value), None)
            if child is None:
                child = {"name": value, "children": []}
                children.append(child)
            current_node["children"] = children
            current_node = child
        current_node["value"] = row["count"]  # Add count as value
    return data

# Convert DataFrame to hierarchical JSON
hierarchical_json = df_to_json(grouped_data)

# Save the JSON to a file
with open("../data/classMonthParkWeather.json", "w") as json_file:
    json.dump(hierarchical_json, json_file, indent=4)